In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

data0 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT00_2021.csv.gz')
data1 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT01_2021.csv.gz')
data2 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT02_2021.csv.gz')
data3 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT03_2021.csv.gz')
data4 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT04_2021.csv.gz')
data5 = pd.read_csv('../../dataset/LODES/ca_od_aux_JT05_2021.csv.gz')

data6 = pd.read_csv('../../dataset/LODES/ca_od_main_JT00_2021.csv.gz')
data7 = pd.read_csv('../../dataset/LODES/ca_od_main_JT01_2021.csv.gz')
data8 = pd.read_csv('../../dataset/LODES/ca_od_main_JT02_2021.csv.gz')
data9 = pd.read_csv('../../dataset/LODES/ca_od_main_JT03_2021.csv.gz')
data10 = pd.read_csv('../../dataset/LODES/ca_od_main_JT04_2021.csv.gz')
data11 = pd.read_csv('../../dataset/LODES/ca_od_main_JT05_2021.csv.gz')

data = pd.concat([data0, data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11])

data_alameda = data[data['w_geocode'].astype(str).str.startswith('6001')]
data_alameda = data_alameda[data_alameda['h_geocode'].astype(str).str.startswith('6001')]

oak_tract = ["600140", "600141", "60014262", "60014261", "60019819", "60019820", "60019832"]

In [2]:
data_oak_work_or_home = data_alameda[
    (data_alameda["w_geocode"].astype(str).str.startswith(tuple(oak_tract)))
    | (data_alameda["h_geocode"].astype(str).str.startswith(tuple(oak_tract)))
]
data_oak_work = data_oak_work_or_home[data_oak_work_or_home["w_geocode"].astype(str).str.startswith(tuple(oak_tract))]
data_oak_work_and_home = data_oak_work[data_oak_work["h_geocode"].astype(str).str.startswith(tuple(oak_tract))]
data_oak_work_and_home["w_geocode"] = data_oak_work_and_home["w_geocode"].apply(lambda x: str(x)[:-4])
data_oak_work_and_home["h_geocode"] = data_oak_work_and_home["h_geocode"].apply(lambda x: str(x)[:-4])
data_oak_flow = data_oak_work_and_home[data_oak_work_and_home["w_geocode"] != data_oak_work_and_home["h_geocode"]]

/var/folders/0z/wc3521k12b5g7pwr410swq240000gq/T/ipykernel_80757/4121379014.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_oak_work_and_home["w_geocode"] = data_oak_work_and_home["w_geocode"].apply(lambda x: str(x)[:-4])
/var/folders/0z/wc3521k12b5g7pwr410swq240000gq/T/ipykernel_80757/4121379014.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_oak_work_and_home["h_geocode"] = data_oak_work_and_home["h_geocode"].apply(lambda x: str(x)[:-4])


In [3]:
data_oak_work.to_csv('work_in_oakland.csv', index=False)
data_oak_work_or_home.to_csv('work_or_home_in_oakland.csv', index=False)
data_oak_work_and_home.to_csv('work_and_home_in_oakland.csv', index=False)
data_oak_flow.to_csv('work_and_home_in_diff_tracts_oakland.csv', index=False)

print(
    "work in Oakland: ",
    data_oak_work.shape[0],
    "\nwork and live in Oakland: ",
    data_oak_work_and_home.shape[0],
    "\nflow in Oakland (work and live in different tracts in Oak): ",
    data_oak_flow.shape[0],
    "\nwork or live in Oakland: ",
    data_oak_work_or_home.shape[0],
)

work in Oakland:  296502 
work and live in Oakland:  155810 
flow in Oakland (work and live in different tracts in Oak):  140109 
work or live in Oakland:  425931


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

oak_tract = ["600140", "600141", "60014262", "60014261", "60019819", "60019820", "60019832"]

In [5]:
# computing n_ij matrix in the block groups and save
data_oak_work_or_home = pd.read_csv('work_or_home_in_oakland.csv')

data_oak_work_or_home = data_oak_work_or_home.sort_values(by=["w_geocode", "h_geocode"])[["w_geocode", "h_geocode", "S000"]]
data_oak_work_or_home["w_in_oak"] = data_oak_work_or_home["w_geocode"].astype(str).str.startswith(tuple(oak_tract))
data_oak_work_or_home["h_in_oak"] = data_oak_work_or_home["h_geocode"].astype(str).str.startswith(tuple(oak_tract))
data_oak_work_or_home.loc[data_oak_work_or_home["w_in_oak"] == False, "w_geocode"] = "oooak"
data_oak_work_or_home.loc[data_oak_work_or_home["h_in_oak"] == False, "h_geocode"] = "oooak"
data_oak_work_or_home = data_oak_work_or_home.groupby(["w_geocode", "h_geocode"]).sum().reset_index()[["w_geocode", "h_geocode", "S000"]]
data_oak_work_or_home = data_oak_work_or_home.pivot(index="w_geocode", columns="h_geocode", values="S000")
data_oak_work_or_home = data_oak_work_or_home.fillna(0).T
data_oak_work_or_home.to_csv('n_ij_block_group_LODES_data.csv')

/var/folders/0z/wc3521k12b5g7pwr410swq240000gq/T/ipykernel_80757/256602077.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'oooak' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_oak_work_or_home.loc[data_oak_work_or_home["w_in_oak"] == False, "w_geocode"] = "oooak"
/var/folders/0z/wc3521k12b5g7pwr410swq240000gq/T/ipykernel_80757/256602077.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'oooak' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_oak_work_or_home.loc[data_oak_work_or_home["h_in_oak"] == False, "h_geocode"] = "oooak"


In [6]:
# computing n_ij matrix in the census tract level and save (note: 402801 and 402802 are combined)
data_oak_work_or_home = pd.read_csv('work_or_home_in_oakland.csv')

data_oak_work_or_home = data_oak_work_or_home.sort_values(by=["w_geocode", "h_geocode"])[["w_geocode", "h_geocode", "S000"]]
data_oak_work_or_home["w_in_oak"] = data_oak_work_or_home["w_geocode"].astype(str).str.startswith(tuple(oak_tract))
data_oak_work_or_home["h_in_oak"] = data_oak_work_or_home["h_geocode"].astype(str).str.startswith(tuple(oak_tract))
data_oak_work_or_home["h_geocode"] = data_oak_work_or_home["h_geocode"].apply(lambda x: str(x)[:-4])
data_oak_work_or_home["w_geocode"] = data_oak_work_or_home["w_geocode"].apply(lambda x: str(x)[:-4])
data_oak_work_or_home.loc[data_oak_work_or_home["w_in_oak"] == False, "w_geocode"] = "oooak"
data_oak_work_or_home.loc[data_oak_work_or_home["h_in_oak"] == False, "h_geocode"] = "oooak"
data_oak_work_or_home = data_oak_work_or_home.groupby(["w_geocode", "h_geocode"]).sum().reset_index()[["w_geocode", "h_geocode", "S000"]]
data_oak_work_or_home = data_oak_work_or_home.pivot(index="w_geocode", columns="h_geocode", values="S000")
data_oak_work_or_home = data_oak_work_or_home.fillna(0).T
data_oak_work_or_home.to_csv('n_ij_census_tract_LODES_data.csv')